In [1]:
import numpy as np
import pandas as pd
import gc

import matplotlib.pyplot as plt
%matplotlib notebook

First - test to see if there are any stars that would be selected by the parallax criterion, but not by the proper motion criterion.

In [2]:
gaia_dir = "/Users/adamamiller/Desktop/PS1_fits/gaia_stars/"
gaia_df = pd.read_hdf(gaia_dir + "parallax_ps1_gaia_mag_pm_plx.h5")
pxl_not_pm = np.where((gaia_df["parallax_over_error"] >= 8) & 
                      (gaia_df["pm_over_error"] < 7.5))

In [3]:
gaia_df.iloc[pxl_not_pm]

,objid,pm_over_error,parallax_over_error,g_mag
109,108000558702683922,4.992758,11.878710,17.258440
146,108000679395059808,5.923615,8.007660,18.647120
184,108000753956420501,5.135121,9.707506,16.752054
191,108000764310144999,6.794067,8.487449,19.225445
201,108000776745494554,7.314615,15.286850,16.418072
236,108000807671276252,5.076812,19.332268,17.940525
247,108000815963484935,4.900018,74.821198,13.828594
285,108000867232342350,3.925954,12.368188,17.717985
286,108000868086405552,5.405010,59.018345,14.168565
318,108000898948784126,7.152426,10.417873,18.761436


There are ~579k sources that satisfy the pxl cut but not the pm cut. These now need to be matched to the pxl cut stars in PS1 to see which (if any) of these stars need to be adjusted in the ZTF database.

In [4]:
pxl_ps1 = pd.read_hdf("parallax_ps1_gaia_cat_merge.h5")

In [5]:
pxl_objid = np.array(gaia_df["objid"].iloc[pxl_not_pm])
pxl_ps1_objid = np.array(pxl_ps1.index)
pxl_in_cat = np.isin(pxl_objid, pxl_ps1_objid)

In [6]:
print("There are {} pxl stars that need to be added to the catalog".format(len(pxl_objid[~pxl_in_cat])))

There are 26658 pxl stars that need to be added to the catalog


In [7]:
pxl_objid[~pxl_in_cat]

array([108001045734103432, 108020229828629738, 108020819165603548, ...,
       107983201466254882, 107990865774634072, 107992840343779445])

In [8]:
del pxl_ps1
del gaia_df
gc.collect()

51

## Get the missing pm stars

In [9]:
gaia_dir = "/Users/adamamiller/Desktop/PS1_fits/gaia_stars/"
pm_df = pd.read_hdf(gaia_dir + "pm_ps1_gaia_mag_pm_plx.h5")

In [10]:
pm_ps1 = pd.read_hdf("pm_ps1_gaia_cat_merge.h5")

In [11]:
pm_objid = np.array(pm_df["objid"])
pm_ps1_objid =np.array(pm_ps1.index)

In [12]:
del pm_df
del pm_ps1
gc.collect()

102

In [13]:
pm_in_cat = np.isin(pm_objid, pm_ps1_objid)

In [14]:
pxl_and_pm_objid = np.append(pm_objid[~pm_in_cat],
                             pxl_objid[~pxl_in_cat])

In [15]:
new_stars_df = pd.DataFrame(pxl_and_pm_objid, columns=["objid"])

In [17]:
new_stars_df.to_hdf("objid_for_gaia_stars_for_ztf_database.h5", "d1")